*Copyright (C) 2021 Intel Corporation*<br>
*SPDX-License-Identifier: BSD-3-Clause*<br>
*See: https://spdx.org/licenses/*

---

# Remote Memory Access

The goal of this tutorial is to show how to enable remote memory access between processes using Lava _RefPorts_. In previous tutorials you have been introduced to _Processes_ which define behavior and _ProcessModels_, which implement the behavior for specific compute resources, e.g., CPU or Loihi Neurocores.

In general processes have only access to its own state and communicate with the enviornment only through messages using ports. Lava also allows certain processes (e.g. those on CPUs) to perform remote memory access of internal states on other processes. Remote memory access between processes is potentially unsafe and should be used with care, but can be very useful in defined cases. One such case would be accessing (read/write) a _Var_ of a _Process_ on a Loihi Neurocore from another _Process_ on the embedded CPU. Thus, _RefPorts_ allow in Lava one _Process_ to access the internal _Vars_ of another _Process_. _RefPorts_ give access to other _Vars_ as if it was an internal _Var_. The read/write of _RefPorts_ are still implemented via messages and can be used across compute resources.


(Image 01)

In this tutorial, we will create minimal Processes and ProcessModels to demonstrate reading and writing of Vars using RefPorts and VarPorts. Furthermore, we will explain the possibilities to connect RefPorts with VarPorts and Vars as well as the difference of explicitly and implicitly created VarPorts.

## Recommended tutorials before starting: 

- [Installing Lava](./tutorial01_installing_lava.ipynb "Tutorial on Installing Lava")
- [Processes](./tutorial02_processes.ipynb "Tutorial on Processes")
- [ProcessModel](./tutorial03_process_models.ipynb "Tutorial on ProcessModels")
- [Execution](./tutorial04_execution.ipynb "Tutorial on Executing Processes")
- [Connecting Processes](./tutorial05_connect_processes.ipynb "Tutorial on Connecting Processes")
- [Hierarchical Processes](./tutorial06_hierarchical_processes.ipynb "Tutorial on Hierarchical Processes")

## Create a minimal _Process_ and _ProcessModel_ with a _RefPort_

The [ProcessModel Tutorial](#tutorial03_process_models.ipynb) walks through the creation of _Processes_ and corresponding _ProcessModels_. In order to demonstrate RefPorts we create a minimal process P1 with a _RefPort_ and a minimal process P2 with a _Var_. We also create the corresponding _ProcessModels_ PyProcModel1 and PyProcModel2 which implement the process P1 and P2. The value of the _Var_ of P2 `var` is initialized with the value 5. The behavior we implement prints out the value of the `var` in P1 every time step, demonstrating the **read** ability of a _RefPort_ `ref`. Afterwards we set the value of `var` by adding the current time step to it and write it with `ref`, demonstrating the **write** abiltity of a _RefPort_. The _RefPort_ `ref` needs to be connected with the _Var_ `var`, before execution.

In [1]:
from lava.magma.core.process.process import AbstractProcess
from lava.magma.core.process.variable import Var
from lava.magma.core.process.ports.ports import RefPort


# A minimal process with a Var and a RefPort
class P1(AbstractProcess):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.ref = RefPort(shape=(1,))

        
# A minimal process with a Var
class P2(AbstractProcess):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.var = Var(shape=(1,), init=5)

#### Create a Python Process Model implementing the Loihi Sync Protocol and requiring a CPU compute resource

In [2]:
import numpy as np

from lava.magma.core.sync.protocols.loihi_protocol import LoihiProtocol
from lava.magma.core.model.py.ports import PyRefPort
from lava.magma.core.model.py.type import LavaPyType
from lava.magma.core.resources import CPU
from lava.magma.core.decorator import implements, requires
from lava.magma.core.model.py.model import PyLoihiProcessModel


# A minimal PyProcModel implementing P1
@implements(proc=P1, protocol=LoihiProtocol)
@requires(CPU)
class PyProcModel1(PyLoihiProcessModel):
    ref: PyRefPort = LavaPyType(PyRefPort.VEC_DENSE, int)

    def pre_guard(self):
        return True

    def run_pre_mgmt(self):
        # Retrieve current value of the Var of P2
        cur_val = self.ref.read()
        print(cur_val)
        
        # Add the current time step to the current value
        new_data = cur_val + self.current_ts
        # Write the new value to the Var of P2
        self.ref.write(new_data)




# A minimal PyProcModel implementing P2
@implements(proc=P2, protocol=LoihiProtocol)
@requires(CPU)
class PyProcModel2(PyLoihiProcessModel):
    var: np.ndarray = LavaPyType(np.ndarray, np.int32)

## Run the _Processes_

In [1]:
from lava.magma.core.run_configs import Loihi1SimCfg
from lava.magma.core.run_conditions import RunSteps

# Create process P1 and P2
proc1 = P1()
proc2 = P2()

# Connect RefPort 'ref' of P1 with Var 'var' of P2
proc1.ref.connect_var(proc2.var)



In [2]:
if __name__ == "__main__":
    # Run the network for 5 time steps
    proc1.run(condition=RunSteps(num_steps=5), run_cfg=Loihi1SimCfg())
    proc1.stop()

## How to learn more?

If you want to find out more about _SubProcessModels_, have a look at the [Lava documentation](https://lava-nc.org/) or dive into the [source code](https://github.com/intel-nrc-ecosystem/lava-core-rethink/blob/main/lava/magma/core/model/sub/model.py).

To receive regular updates on the latest developments and releases of the Lava Software Framework please subscribe to [our newsletter](http://eepurl.com/hJCyhb).